<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5ef075e31a2a9329664b2679dcd5dcb590c921ff9376bf818ae1d1644c37d12c
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-30 13:07:57
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.36 C
-------------------
Today PnL: 2.53 K (0.02%)
Current PnL: -23.09 L (-15.53%)
CY Booked + Current PnL: -11.65 L (-7.84%)
-------------------
Total profit:  1.10 L
Total loss:  -24.20 L
-------------------
Total Booked + Current PnL: 15.33 L (12.59%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.38%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.96 L (65.17%)
Deployed:  1.22 C
Current:  1.36 C
CAGR/XIRR %: 6.91%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-43.66,61.0,H-MC,12.24,112572.0,12068.0,17043.0,4.16,12.01,15.14,28.96,79.0,0.71,0.82,50.05,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.60,37.0,M-SC,2.45,83408.0,-17369.0,17466.0,0.28,-17.24,20.94,0.09,245.0,-0.99,0.61,9.58,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.48,47.0,H-MC,3.42,189000.0,24330.0,18995.0,-0.30,14.78,10.05,26.31,89.0,1.28,1.39,23.94,X40N,NTT,AC
84,VOLTAS,1530.00,-1.92,48.0,H-MC,2.88,205800.0,14058.0,23708.0,2.37,7.33,11.52,19.69,99.0,0.59,1.51,14.63,XY25,NTT,AC
42,ITC,452.00,-39.19,42.0,H-LC,1.95,196627.0,-3511.0,24401.0,-1.26,-1.75,12.41,10.44,4.0,-0.14,1.44,3.71,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1234.93,69.0,M-SC,14.24,181587.0,444.0,45015.0,6.03,0.24,24.79,25.10,235.0,0.01,1.33,45.52,XY24,NTT,HEALTHCARE
50,NATIONALUM,244.55,-43.66,61.0,H-MC,12.24,112572.0,12068.0,17043.0,4.16,12.01,15.14,28.96,79.0,0.71,0.82,50.05,MH,ATH,METALS
79,UNIONBANK,163.00,-13.18,57.0,M-LC,7.12,157463.0,16623.0,27871.0,3.29,11.80,17.70,31.59,66.0,0.60,1.15,41.42,XY24,NTT,BANKS
11,BANKINDIA,190.00,-26.71,64.0,H-MC,8.90,188497.0,8689.0,103730.0,3.19,4.83,55.03,62.52,88.0,0.08,1.38,40.57,XR,NTT,BANKS
52,PGHH,17973.08,-27.11,65.0,H-MC,3.21,210885.0,10065.0,58710.0,3.01,5.01,27.84,34.25,80.0,0.17,1.55,10.47,X40,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.1,-45.66,20.0,H-SC,14.27,82260.0,-20240.0,60839.0,-3.81,-19.75,73.96,39.61,114.0,-0.33,0.60,0.00,AR,ATH,MISC
85,WHIRLPOOL,2270.0,-46.98,25.0,M-SC,1.27,89520.0,-1978.0,80729.0,-3.16,-2.16,90.18,86.07,223.0,-0.02,0.66,30.00,XR,NTT,DURABLES
38,INDIGOPNTS,1408.0,99.78,28.0,M-SC,1.55,131018.0,-43541.0,43577.0,-3.12,-24.94,33.26,0.02,221.0,-1.00,0.96,13.96,OX40N,NTT,PAINTS
26,GREENPANEL,537.0,192.64,39.0,M-SC,0.98,130638.0,-52440.0,131422.0,-3.01,-28.64,100.60,43.14,230.0,-0.40,0.96,22.88,XY24,NTT,MISC
81,VAIBHAVGBL,521.0,46.56,43.0,H-SC,5.81,127295.0,-55480.0,166553.0,-2.92,-30.35,130.84,60.77,125.0,-0.33,0.93,14.01,XR,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1234.93,69.0,M-SC,14.24,181587.0,444.0,45015.0,6.03,0.24,24.79,25.10,235.0,0.01,1.33,45.52,XY24,NTT,HEALTHCARE
37,INDIAMART,4850.92,-55.57,24.0,H-SC,9.97,124243.0,907.0,132853.0,-0.95,0.74,106.93,108.45,119.0,0.01,0.91,23.25,AR,ATH,MISC
35,ICICIPRULI,790.00,-21.80,43.0,H-MC,2.21,134493.0,-1295.0,44046.0,1.64,-0.95,32.75,31.48,107.0,-0.03,0.99,11.05,X40,ATH,INSURANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,97.20,56.0,H-SC,10.80,124308.0,-18171.0,37690.0,-0.09,-12.75,30.32,13.70,163.0,-0.48,0.91,46.72,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.60,37.0,M-SC,2.45,83408.0,-17369.0,17466.0,0.28,-17.24,20.94,0.09,245.0,-0.99,0.61,9.58,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,51.0,H-SC,1.54,221463.0,-48204.0,84532.0,-0.21,-17.88,38.17,13.47,138.0,-0.57,1.62,12.01,XY24,NTT,PAINTS
18,CERA,9475.0,-22.43,38.0,H-SC,1.59,141484.0,-34419.0,76444.0,-0.11,-19.57,54.03,23.89,149.0,-0.45,1.04,22.19,OX40N,NTT,CERAMICS
66,SIS,528.0,2067.36,38.0,H-SC,4.01,87465.0,-23567.0,47179.0,-0.64,-21.23,53.94,21.26,156.0,-0.50,0.64,18.17,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1234.93,69.0,M-SC,14.24,181587.0,444.0,45015.0,6.03,0.24,24.79,25.10,235.0,0.01,1.33,45.52,XY24,NTT,HEALTHCARE
60,SAIL,228.00,45.95,58.0,M-MC,11.20,230559.0,5597.0,162060.0,1.66,2.49,70.29,74.53,192.0,0.03,1.69,35.54,XY24,BTT,STEEL
32,HINDZINC,730.22,27.88,64.0,M-LC,3.39,208126.0,3050.0,109516.0,2.64,1.49,52.62,54.89,52.0,0.03,1.53,25.94,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.48,47.0,H-MC,3.42,189000.0,24330.0,18995.0,-0.30,14.78,10.05,26.31,89.0,1.28,1.39,23.94,X40N,NTT,AC
84,VOLTAS,1530.00,-1.92,48.0,H-MC,2.88,205800.0,14058.0,23708.0,2.37,7.33,11.52,19.69,99.0,0.59,1.51,14.63,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-43.66,61.0,H-MC,12.24,112572.0,12068.0,17043.0,4.16,12.01,15.14,28.96,79.0,0.71,0.82,50.05,MH,ATH,METALS
11,BANKINDIA,190.00,-26.71,64.0,H-MC,8.90,188497.0,8689.0,103730.0,3.19,4.83,55.03,62.52,88.0,0.08,1.38,40.57,XR,NTT,BANKS
32,HINDZINC,730.22,27.88,64.0,M-LC,3.39,208126.0,3050.0,109516.0,2.64,1.49,52.62,54.89,52.0,0.03,1.53,25.94,X5K,ATH,METALS
37,INDIAMART,4850.92,-55.57,24.0,H-SC,9.97,124243.0,907.0,132853.0,-0.95,0.74,106.93,108.45,119.0,0.01,0.91,23.25,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-46.98,25.0,M-SC,1.27,89520.0,-1978.0,80729.0,-3.16,-2.16,90.18,86.07,223.0,-0.02,0.66,30.00,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JIOFIN,387.00,-6.90,26.0,H-LC,13.02,253012.0,-14353.0,79800.0,0.00,-5.37,31.54,24.48,48.0,-0.18,1.85,46.65,XY24,BTT,FINANCE
5,ASIANPAINT,3465.66,-18.29,28.0,H-LC,6.14,206351.0,-45417.0,98636.0,0.10,-18.04,47.80,21.13,27.0,-0.46,1.51,11.00,X40,ATH,PAINTS
74,TCS,4476.75,-31.26,30.0,H-LC,12.27,273333.0,-72623.0,147490.0,0.40,-20.99,53.96,21.64,1.0,-0.49,2.00,0.40,X40,ATH,IT
83,VBL,671.64,-21.55,32.0,H-LC,8.84,282698.0,-33144.0,145816.0,-1.90,-10.49,51.58,35.67,5.0,-0.23,2.07,1.91,X40N,ATH,FMCG
19,CIPLA,1795.00,-23.21,37.0,H-LC,3.98,204952.0,452.0,40970.0,0.54,0.22,19.99,20.25,10.0,0.01,1.50,8.97,X40N,BTT,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.23,43.0,H-LC,1.61,230220.0,-15931.0,95150.0,-0.81,-6.47,41.33,32.18,16.0,-0.17,1.69,27.24,X200,ATH,IT
42,ITC,452.0,-39.19,42.0,H-LC,1.95,196627.0,-3511.0,24401.0,-1.26,-1.75,12.41,10.44,4.0,-0.14,1.44,3.71,X40,NTT,FMCG
65,SIEMENS,4671.5,-3.25,42.0,H-LC,2.40,155590.0,-30505.0,77982.0,0.50,-16.39,50.12,25.51,15.0,-0.39,1.14,14.66,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.79,39.0,H-LC,3.76,212987.0,2621.0,26155.0,-0.55,1.25,12.28,13.68,37.0,0.10,1.56,17.97,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-23.21,37.0,H-LC,3.98,204952.0,452.0,40970.0,0.54,0.22,19.99,20.25,10.0,0.01,1.50,8.97,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,30.0,H-LC,12.27,273333.0,-72623.0,147490.0,0.40,-20.99,53.96,21.64,1.0,-0.49,2.00,0.40,X40,ATH,IT
80,UNITDSPR,1644.00,-15.08,42.0,H-LC,7.21,222706.0,-11438.0,60064.0,-1.05,-4.88,26.97,20.77,86.0,-0.19,1.63,1.69,X40N,NTT,BREWERIES
83,VBL,671.64,-21.55,32.0,H-LC,8.84,282698.0,-33144.0,145816.0,-1.90,-10.49,51.58,35.67,5.0,-0.23,2.07,1.91,X40N,ATH,FMCG
42,ITC,452.00,-39.19,42.0,H-LC,1.95,196627.0,-3511.0,24401.0,-1.26,-1.75,12.41,10.44,4.0,-0.14,1.44,3.71,X40,NTT,FMCG
30,HCLTECH,1943.91,-3.66,38.0,H-LC,10.12,220224.0,-21700.0,84962.0,1.10,-8.97,38.58,26.15,8.0,-0.26,1.61,4.07,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.25,42.0,H-LC,2.40,155590.0,-30505.0,77982.0,0.50,-16.39,50.12,25.51,15.0,-0.39,1.14,14.66,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,42.0,H-LC,1.95,196627.0,-3511.0,24401.0,-1.26,-1.75,12.41,10.44,4.0,-0.14,1.44,3.71,X40,NTT,FMCG
19,CIPLA,1795.00,-23.21,37.0,H-LC,3.98,204952.0,452.0,40970.0,0.54,0.22,19.99,20.25,10.0,0.01,1.50,8.97,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-18.29,28.0,H-LC,6.14,206351.0,-45417.0,98636.0,0.10,-18.04,47.80,21.13,27.0,-0.46,1.51,11.00,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.79,39.0,H-LC,3.76,212987.0,2621.0,26155.0,-0.55,1.25,12.28,13.68,37.0,0.10,1.56,17.97,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,30.0,H-LC,12.27,273333.0,-72623.0,147490.0,0.40,-20.99,53.96,21.64,1.0,-0.49,2.00,0.40,X40,ATH,IT
40,INFY,2275.00,-21.03,39.0,H-LC,8.20,308041.0,-4847.0,176538.0,0.31,-1.55,57.31,54.87,3.0,-0.03,2.26,5.00,X40,BTT,IT
42,ITC,452.00,-39.19,42.0,H-LC,1.95,196627.0,-3511.0,24401.0,-1.26,-1.75,12.41,10.44,4.0,-0.14,1.44,3.71,X40,NTT,FMCG
83,VBL,671.64,-21.55,32.0,H-LC,8.84,282698.0,-33144.0,145816.0,-1.90,-10.49,51.58,35.67,5.0,-0.23,2.07,1.91,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,46.0,H-LC,7.68,249811.0,-11808.0,131026.0,0.60,-4.51,52.45,45.57,7.0,-0.09,1.83,6.30,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,41.0,L-SC,40.19,77760.0,-35789.0,75839.0,0.17,-31.52,97.53,35.27,268.0,-0.47,0.57,90.32,XR,NTT,HOTELS
49,MASFIN,398.61,-20.96,35.0,H-SC,12.28,89865.0,-8115.0,29718.0,-1.02,-8.28,33.07,22.05,152.0,-0.27,0.66,30.63,XR,ATH,FINANCE
61,SAMMAANCAP,326.00,-180.68,75.0,M-SC,7.07,93588.0,-8622.0,102011.0,1.11,-8.44,109.00,91.37,208.0,-0.08,0.69,51.92,XY25,NTT,FINANCE
50,NATIONALUM,244.55,-43.66,61.0,H-MC,12.24,112572.0,12068.0,17043.0,4.16,12.01,15.14,28.96,79.0,0.71,0.82,50.05,MH,ATH,METALS
78,UJJIVANSFB,60.00,97.20,56.0,H-SC,10.80,124308.0,-18171.0,37690.0,-0.09,-12.75,30.32,13.70,163.0,-0.48,0.91,46.72,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1234.93,69.0,M-SC,14.24,181587.0,444.0,45015.0,6.03,0.24,24.79,25.10,235.0,0.01,1.33,45.52,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-180.68,75.0,M-SC,7.07,93588.0,-8622.0,102011.0,1.11,-8.44,109.00,91.37,208.0,-0.08,0.69,51.92,XY25,NTT,FINANCE
50,NATIONALUM,244.55,-43.66,61.0,H-MC,12.24,112572.0,12068.0,17043.0,4.16,12.01,15.14,28.96,79.0,0.71,0.82,50.05,MH,ATH,METALS
60,SAIL,228.00,45.95,58.0,M-MC,11.20,230559.0,5597.0,162060.0,1.66,2.49,70.29,74.53,192.0,0.03,1.69,35.54,XY24,BTT,STEEL
32,HINDZINC,730.22,27.88,64.0,M-LC,3.39,208126.0,3050.0,109516.0,2.64,1.49,52.62,54.89,52.0,0.03,1.53,25.94,X5K,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.24
1,25,43.72
2,50,75.07


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.64
LC    32.53
MC    23.83
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.38
X40      14.82
X40N     11.36
XR       11.07
XY25     10.27
AR        8.18
OX40N     7.86
X200      1.69
MH        1.64
X5K       1.53
SR        1.20
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.11
H-LC    25.92
H-MC    20.85
M-SC    13.09
M-LC     5.56
M-MC     2.67
L-SC     1.44
L-LC     1.05
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.09,-6.81,42.06
IT,12.71,-21.21,87.30
FINANCE,10.10,-18.92,70.80
MISC,6.93,-21.59,89.18
BANKS,6.32,-15.76,76.66
PAINTS,5.61,-20.48,38.45
ELECTRICAL,5.48,-9.96,48.54
INSURANCE,3.80,-6.97,46.96
AC,3.61,1.51,14.98


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3233999.0,22
XR,1399510.0,14
AR,1215980.0,9
X40,825847.0,10
X40N,654792.0,9
OX40N,573004.0,10
XY25,439563.0,7
SR,258712.0,2
X5K,109516.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3821221.0,29
M-SC,1460557.0,17
H-LC,1281740.0,15
H-MC,1262387.0,15
M-LC,391171.0,4
M-MC,314111.0,2
L-SC,263607.0,3
L-MC,60980.0,1
L-LC,40305.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1249565.0      6
           AR         835769.0      5
M-SC       XY24       834684.0      7
H-SC       XR         818962.0      7
H-MC       XY24       585501.0      4
H-LC       X40        532027.0      5
H-SC       X40N       321055.0      4
M-MC       XY24       314111.0      2
H-SC       OX40N      264195.0      4
           SR         258712.0      2
H-LC       X40N       246850.0      3
H-MC       X40        217331.0      4
H-LC       AR         209008.0      2
H-MC       XY25       178342.0      2
L-SC       XR         174759.0      2
M-SC       AR         171203.0      2
M-LC       XY24       170338.0      2
M-SC       OX40N      146408.0      4
           XR         129762.0      2
H-LC       XY25       118905.0      3
M-LC       XR         111317.0      1
           X5K        109516.0      1
H-MC       XR         103730.0      1
M-SC       XY25       102011.0      1
H-LC       X200        95150.0      1
L-SC       OX40N       88848.0      1
H-MC       X40N        86887.0      2
H-LC       XY24        79800.0      1
M-SC       X40         76489.0      1
H-MC       OX40N       73553.0      1
H-SC       MH          72963.0      1
L-MC       XR          60980.0      1
L-LC       XY25        40305.0      1
H-MC       MH          17043.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
